## 用栈实现队列

题目链接：https://leetcode.cn/problems/implement-queue-using-stacks/description/

本题要求使用栈实现队列，关键点在于如何使用栈这种 LIFO 的特性模拟队列这种 FIFO 的数据结构特性。

解题思路：

由于 python 中并没有严格的栈数据结构，所以，我沿用了上一个章节的定义好的栈。

我们可以定义两个栈，`in_stack` 和 `out_stack`，用两个栈来模拟队列的入队和出队操作。

入队操作（`push`）相对简单，我们只需要将新元素压入 `in_stack` 即可。

出队操作（`pop`）和查看队首元素（`peek`）则稍微复杂一些。由于栈是后进先出的数据结构，而我们需要实现的是先进先出的队列，因此在执行出队或查看队首元素之前，我们需要先将 `in_stack` 中的所有元素依次弹出并压入 `out_stack`。这样，`in_stack` 中最先进入的元素就会被移到 `out_stack` 的栈顶，从而实现了先进先出的特性。这个过程在代码中通过下面的 `in2out` 方法实现。

`in2out` 方法的作用是将 `in_stack` 中的所有元素转移到 `out_stack` 中，同时保证元素的顺序不变。这是通过将 `in_stack` 中的元素依次弹出并压入 `out_stack` 来实现的。由于栈的后进先出特性，这样做正好将元素的顺序反转了两次，从而保持了原始顺序。需要注意的是，我们只在 `out_stack` 为空时才调用 `in2out` 方法进行元素转移。当 `out_stack` 不为空时，`peek` 方法与 `pop` 方法都可以直接对 `out_stack` 进行操作。这是为了遵循次序性。

完整代码如下：

In [1]:
# https://leetcode.cn/problems/implement-queue-using-stacks/description/
# 232. 用栈实现队列
class Stack:
    def __init__(self):
        self.stack = []

    def push(self, item):
        self.stack.append(item)

    def pop(self):
        if self.is_empty():
            raise Exception("Stack is empty")
        return self.stack.pop()

    def peek(self):
        if self.is_empty():
            raise Exception("Stack is empty")
        return self.stack[-1]

    def is_empty(self):
        return len(self.stack) == 0

    def size(self):
        return len(self.stack)


class MyQueue:

    def __init__(self):
        self.in_stack = Stack()
        self.out_stack = Stack()

    def push(self, x: int) -> None:
        self.in_stack.push(x)

    def pop(self) -> int:
        if self.empty():
            raise Exception("Queue is empty")
        else:
            if self.out_stack.is_empty():
                self.in2out()
            return self.out_stack.pop()

    def peek(self) -> int:
        if self.empty():
            raise Exception("Queue is empty")
        else:
            if self.out_stack.is_empty():
                self.in2out()
            return self.out_stack.peek()

    def empty(self) -> bool:
        return self.in_stack.is_empty() and self.out_stack.is_empty()

    def in2out(self):
        while not self.in_stack.is_empty():
            self.out_stack.push(self.in_stack.pop())


# test
myQueue = MyQueue()
myQueue.push(1)  # queue is: [1]
myQueue.push(2)  # queue is: [1, 2] (leftmost is front of the queue)
print(myQueue.peek())  # return 1
print(myQueue.pop())  # return 1, queue is [2]
print(myQueue.pop())  # return 2, queue is []
print(myQueue.empty())  # return True


1
1
2
True


在这个用两个栈实现队列的算法中，各个操作的时间复杂度如下：

1. `push` 操作：
    将一个元素压入 `in_stack` 的时间复杂度是 `O(1)`，因为这是一个简单的栈操作。
2. `pop` 和 `peek` 操作：
    在最坏的情况下，当 `out_stack` 为空时，我们需要将 `in_stack` 中的所有元素转移到 `out_stack` 中。这个转移操作的时间复杂度是 `O(n)`，其中 `n` 是 `in_stack` 中的元素数量。然而，这个操作并不是每次 `pop` 或 `peek` 都会发生，而是只在 `out_stack` 为空时发生。一旦 `out_stack` 被填充，后续的 `pop` 和 `peek` 操作就只需要在 `out_stack` 上执行，时间复杂度降为 `O(1)`，直到 `out_stack` 再次变空。
3. `empty` 操作：
    检查两个栈是否为空的时间复杂度是 `O(1)`，因为只需要检查两个栈的大小。

从均摊分析的角度来看，每个元素最多只会被压入和弹出每个栈各一次。因此，虽然单次操作可能达到 `O(n)` 的时间复杂度，但从一个元素的完整生命周期来看，其均摊时间复杂度仍然是 `O(1)`。这是因为每个元素只会经历一次从 `in_stack` 到 `out_stack` 的转移，而转移的成本可以被平摊到该元素的入队和出队操作上。

综上所述，虽然某些操作在最坏情况下可能达到 `O(n)` 的时间复杂度，但从均摊分析的角度来看，每个操作的平均时间复杂度可以视为 `O(1)`。

## 用队列实现栈

题目链接：https://leetcode.cn/problems/implement-stack-using-queues/description/

本题要求使用队列实现栈。

解题思路：

由于 python 中并没有严格的队列数据结构，所以，我沿用了上一个章节的定义好的队列。

入栈操作（`push`）的实现：

在栈中，最后加入的元素应该是第一个被取出的，这体现了栈的后进先出（LIFO）特性。然而，队列是先进先出（FIFO）的数据结构，即最早加入的元素会最先被取出。因此，为了使用队列实现栈的`push`操作，我们需要一种方法来反转队列中元素的顺序，使得每次`push`的元素都能被放在队列的前面，这样在模拟栈的`pop`操作时，就能保证最后`push`的元素最先被`pop`出来。

具体做法：
1. 记录当前队列的大小：在将新元素加入队列之前，我们先记录下当前队列中元素的数量（假设为`size`）。
2. 新元素入队：将新元素加入队列的尾部。
3. 元素重排：然后，我们将队列前面的`size`个元素依次出队，并立即将它们重新入队。这样做的目的是将新加入的元素“挤”到队列的前面去，同时保持原有元素的相对顺序不变。
4. 完成入栈：经过上述步骤，新加入的元素现在位于队列的最前面，这样就模拟了栈的`push`操作。

这种方法的代价是，每次`push`操作都需要移动队列中已有的所有元素，因此时间复杂度是`O(n)`，其中`n`是`push`操作前栈中的元素数量。

出栈操作（`pop`）和查看栈顶元素（`top`）的实现则相对简单。由于我们已经通过入栈操作确保了最后一个入栈的元素总是在队列的前面，因此我们可以直接调用队列的出队（`dequeue`）和查看队首元素（`peek`）操作来实现栈的出栈和查看栈顶元素功能。这两个操作的时间复杂度都是 `O(1)`。

In [ ]:
# https://leetcode.cn/problems/implement-stack-using-queues/
# 225. 用队列实现栈
from collections import deque


class Queue:
    def __init__(self):
        self.queue = deque()

    def peek(self):
        if not self.is_empty():
            peek_val = self.queue.popleft()
            self.queue.appendleft(peek_val)
            return peek_val
        else:
            raise Exception('Queue is empty')

    def enqueue(self, item):
        self.queue.append(item)

    def dequeue(self):
        if not self.is_empty():
            return self.queue.popleft()
        else:
            raise Exception('Queue is empty')

    def is_empty(self):
        return len(self.queue) == 0

    def size(self):
        return len(self.queue)


class MyStack:

    def __init__(self):
        self.q = Queue()

    def push(self, x: int) -> None:
        size = self.q.size()
        self.q.enqueue(x)
        i = 0
        while i < size:
            self.q.enqueue(self.q.dequeue())
            i += 1

    def pop(self) -> int:
        if self.empty():
            raise Exception('Stack is empty')
        return self.q.dequeue()

    def top(self) -> int:
        if self.empty():
            raise Exception('Stack is empty')
        return self.q.peek()

    def empty(self) -> bool:
        return self.q.is_empty()


# test
stack = MyStack()
stack.push(1)
stack.push(2)
print(stack.top())  # 2
print(stack.pop())  # 2
print(stack.pop())  # 1
print(stack.empty())  # True
